## Trackers
The nuts-and-bolts implementation of the bridge between `traitlets` and the on-disk representation.

In [ ]:
import ipywidgets as W, traitlets as T, pathlib as P, git as G, yaml, json, ipywidgets.embed as E
from tornado.ioloop import IOLoop
from tornado.concurrent import run_on_executor
from concurrent.futures import ThreadPoolExecutor
from wxyz.dvcs import Watcher
from pathlib import Path

### Base Tracker

In [ ]:
class _Tracker(W.Widget):
    executor = ThreadPoolExecutor(max_workers=1)
    tracked_widget = T.Instance(W.Widget)
    tracked_traits = W.trait_types.TypedTuple(T.Unicode(), allow_none=True, help="trait names to track (default all)")
    path = T.Instance(P.Path)
    encoding = T.Unicode("utf-8")
    change_number = T.Int(0)
    __extension__ = None
    
    async def on_user_change(self):
        raise NotImplementedError("tracker subclass must implement `async on_user_change`")
    
    async def on_file_change(self):
        raise NotImplementedError("tracker subclass must implement `async on_user_change`")
    
    def _on_user_change(self, change):
        try:
            IOLoop.current().add_callback(self.on_user_change)
            self.change_number += 1
        except:
            pass
        
    def _on_file_change(self, change):
        try:
            IOLoop.current().add_callback(self.on_file_change)
        except:
            pass

    @T.observe("tracked_widget")
    def _changed_tracked_widget(self, change):
        if change.old is not None and isinstance(change.old, W.Widget):
            change.old.unobserve(self._on_user_change, self.tracked_traits or T.All)
        if change.new is not None:
            change.new.observe(self._on_user_change, self.tracked_traits or T.All)
            self._on_file_change(None)
            
    @T.observe("tracked_traits")
    def _changed_tracked_traits(self, change):
        if self.tracked_widget is None or not isinstance(self.tracked_widget, W.Widget):
            return
        if change.old is None or isinstance(change.old, tuple):
            self.tracked_widget.unobserve(self._on_user_change, change.old or T.All)
        if change.new is None or isinstance(change.new, tuple):
            self.tracked_widget.observe(self._on_user_change, change.new or T.All)
            self._on_file_change(None)
    
    @classmethod
    def _detect_tracker(cls, path, base_cls=None):
        """ naive tracker finder... needs something cleverer
        """
        base_cls = base_cls or _Tracker
        
        if base_cls.__extension__ and path.name.endswith(base_cls.__extension__):
            return base_cls
        
        subclasses = base_cls.__subclasses__()
        
        for sub_cls in subclasses:
            if sub_cls.__extension__ and path.name.endswith(sub_cls.__extension__):
                return sub_cls
        
        for sub_cls in subclasses:
            subsub = cls._detect_tracker(path, sub_cls)
            if subsub:
                return subsub
        
        return None

### Dict Tracker

Store as a dictionary (aka key/value pair, map, hash), or generally vanilla JSON-compatible (no timestamps, etc).

In [ ]:
class _DictTracker(_Tracker):
    def dict_from_widget(self):
        d = {}
        w = self.tracked_widget
        if w is not None:
            s = E.dependency_state(w)[w.comm.comm_id]["state"]
            for k in self.tracked_traits or list(s):
                d[k] = s.get(k, None)
        return d
    
    def widget_from_dict(self, content):
         with self.tracked_widget.hold_trait_notifications():
            for trait in self.tracked_traits or self.tracked_widget.trait_names():
                if trait in content:
                    new_value = content[trait]
                    old_value = getattr(self.tracked_widget, trait) 
                    if new_value != old_value:
                        setattr(self.tracked_widget, trait, new_value)

### JSON Dict Tracker
While not the most naively revision-control-aware format, JSON is widely distributed.

In [ ]:
class JSONDictTracker(_DictTracker):
    __extension__ = ".json"
    
    @run_on_executor
    def on_user_change(self):
        self.path.write_text(
            json.dumps(self.dict_from_widget(), sort_keys=True, indent=True), 
            encoding=self.encoding
        )
    
    @run_on_executor
    def on_file_change(self):
        if not self.path.exists():
            return
        content = self.widget_from_dict(json.loads(self.path.read_text()))

### YAML Dict Tracker
YAML is somewhat more flexible for change control, but whitespace can be a bummer

In [ ]:
class YAMLDictTracker(_DictTracker):
    __extension__ = ".yaml"
    
    @run_on_executor
    def on_user_change(self):
        self.path.write_text(
            yaml.safe_dump(self.dict_from_widget(), default_flow_style=False), 
            encoding=self.encoding
        )
    
    @run_on_executor
    def on_file_change(self):
        if not self.path.exists():
            return

        self.widget_from_dict(yaml.safe_load(self.path.read_text()))